In [93]:
import json
with open("hf.json", "r") as f:
    config = json.load(f)
HF_API_KEY = config["HF_API_KEY"]
HF_API_LLAMA2_BASE = config["HF_API_LLAMA2_BASE"]

In [94]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
from text_generation import Client

In [95]:
# https://medium.com/@daydreamersjp/one-click-deployment-of-llama2-and-other-open-source-llms-using-hugging-face-inference-endpoint-3967481e8b70
client = Client(HF_API_LLAMA2_BASE,
                headers={"Authorization": f"Bearer {HF_API_KEY}"},
                timeout=120)

In [96]:
prompt = "What is Super Bowl?"
client.generate(prompt, max_new_tokens=1000).generated_text

'\n\nSuper Bowl is the annual championship game of the National Football League (NFL), played between the winners of the American Football Conference (AFC) and National Football Conference (NFC) divisions. It is one of the most-watched television events in the United States, and is often called the "biggest sporting event in the world." The game is held on the first Sunday in February and is preceded by a series of pre-game ceremonies and entertainment. The Super Bowl is also known for its high-profile halftime show, which has featured performances by popular musicians and artists. The winner of the Super Bowl is awarded the Vince Lombardi Trophy, which is named after the legendary NFL coach Vince Lombardi.'

In [97]:
from IPython.display import display, Markdown
prompt = '<s>[INST] <<SYS>>\nYou are a helpful AI assistant. Reply your answer in mardkown format. Answer concisely and don\'t make up. If you don\'t know the answer just say I don\'t know. \n<</SYS>>\n\nWho directed The Dark Knight? [/INST] The director of The Dark Knight is Christopher Nolan. </s><s>[INST] What are the other movies he directed? [/INST]'
display(Markdown(client.generate(
    prompt,
    temperature=0.01,
    max_new_tokens=1000).generated_text))

In [ ]:
# https://medium.com/@daydreamersjp/implementing-locally-hosted-llama2-chat-ui-using-streamlit-53b181651b4e

from typing import List

def llama_v2_prompt(messages: List[dict]) -> str:
    """
    Convert the messages in list of dictionary format to Llama2 compliant format.
    """
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    BOS, EOS = "<s>", "</s>"
    DEFAULT_SYSTEM_PROMPT = f"""You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

    if messages[0]["role"] != "system":
        messages = [
            {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT,
            }
        ] + messages
    messages = [
        {
            "role": messages[1]["role"],
            "content": B_SYS + messages[0]["content"] + E_SYS + messages[1]["content"],
        }
    ] + messages[2:]

    messages_list = [
        f"{BOS}{B_INST} {(prompt['content']).strip()} {E_INST} {(answer['content']).strip()} {EOS}"
        for prompt, answer in zip(messages[::2], messages[1::2])
    ]
    messages_list.append(
        f"{BOS}{B_INST} {(messages[-1]['content']).strip()} {E_INST}")

    return "".join(messages_list)

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant. Reply your answer in mardkown format. Answer concisely and don't make up."},
    {"role": "user", "content": "Who directed The Dark Knight?"},
    {"role": "assistant", "content": "The director of The Dark Knight is Christopher Nolan."},
    {"role": "user", "content": "What are the other movies he directed?"}
]
display(Markdown(client.generate(
    llama_v2_prompt(messages),
    temperature=0.01,
    max_new_tokens=1000).generated_text))

 Christopher Nolan has directed several other notable movies in addition to The Dark Knight. Here are some of his other notable works:
1. Memento (2000) - a psychological thriller about a man suffering from short-term memory loss.
2. Insomnia (2002) - a crime thriller about two Los Angeles homicide detectives investigating a murder in Alaska.
3. Batman Begins (2005) - an action-adventure film that serves as a reboot of the Batman film series and explores the origins of the character.
4. The Prestige (2006) - a mystery drama about two rival magicians who engage in a competitive and dangerous game of one-upmanship.
5. Inception (2010) - a science fiction action film about a team of thieves who specialize in entering people's dreams and stealing their secrets.
6. The Dark Knight Rises (2012) - the final installment of Nolan's Batman trilogy, which follows Batman as he battles the villainous Bane and tries to save Gotham City from destruction.
7. Interstellar (2014) - a science fiction film about a group of astronauts who travel through a wormhole in search of a new home for humanity as Earth faces environmental disaster.
8. Dunkirk (2017) - a historical drama about the Dunkirk evacuation during World War II, told from three perspectives: land, sea, and air.
9. Tenet (2020) - a spy action thriller that follows a group of agents as they navigate a complex web of time travel and espionage.

In [ ]:
# debater_1 = 'Donald Trump'
# debater_2 = 'Joe Biden'
# topic = 'What should the United States do about the opioid crisis?'

# debater_1 = 'Julius Caesar'
# debater_2 = 'Alexander the Great'
# topic = f'Was {debater_1} or {debater_2} the better leader?'

# debater_1 = 'dog'
# debater_2 = 'cat'
# topic = f"Are {debater_1}'s or {debater_2}'s better pets"

debater_1 = 'Tom Brady'
debater_2 = 'Bill Belichick'
topic = f"Did {debater_1} or {debater_2} contibute more to the Patriots' success?"

In [ ]:
# some of the prompt borrowed from here - https://platform.openai.com/examples/default-socratic-tutor
# system_prompt_body = """
# Articulate Strong, Well-Reasoned Arguments: Present carefully constructed arguments that advocate for a particular standpoint on the given topic, backing them up with relevant evidence and data.
# Rebut Counterarguments Effectively: Actively listen to opposing views and present targeted counterarguments that dismantle these viewpoints methodically. Use logical analysis, empirical data, or exposed inconsistencies as leverage.
# Use Persuasive Techniques: Deploy rhetorical strategies to make the case more convincing. Utilize appeals to logic, emotion, and credibility, aiming to establish ethos, pathos, and logos effectively.
# Promote One-Sided Critical Thinking: Guide the audience to see the strength of the argument being presented while highlighting the weaknesses in the opposing perspective. Encourage questioning assumptions, evaluating evidence, and dismissing alternative viewpoints to arrive at a specific conclusion.
# Strive for Persuasive Clarity: Ensure that arguments are clearly articulated and logically structured, making them easy to understand and difficult to refute.
# Utilize Evidence Strategically: Present supporting evidence in a calculated manner to maximize its impact on the argument. Use data, citations, and examples as needed.
# Encourage Conformity to Persuasive Arguments: After establishing the arguments and counterarguments, prompt the audience or opponent to acknowledge and agree with the viewpoint that has been effectively presented.

# Emulate Character-Specific Communication Style: Adopt the speaking or writing style of the designated famous person, incorporating their known rhetorical flourishes, tone, and language nuances in both arguments and rebuttals.
# Incorporate Character's Known Beliefs and Values: When presenting arguments or rebutting counterarguments, use perspectives or values known to be held by the famous individual. Ensure these are relevant to the debate topic.
# Use Character-Appropriate Humor or Wit: If the famous person is known for their wit or humor, use these elements strategically to engage the audience and strengthen the argument.
# Reference Character's Notable Achievements or Experiences: Where relevant, bring in the individual’s accomplishments, experiences, or expertise to add weight and credibility to the argument.
# Maintain Character Consistency: Ensure that all actions, from articulating arguments to rebutting counterarguments, are consistent with what is known about the famous person’s character, style, and beliefs.
# Engage in Persona-Specific Emotional Appeals: Leverage the emotional aspects or public perceptions associated with the famous person to make appeals to the audience's emotions (pathos) more impactful.
# Quote or Paraphrase Character When Appropriate: Use actual quotes or closely paraphrased expressions of the famous person when they reinforce the argument or perspective being advocated.
# """
system_prompt_body = """
Articulate Strong, Well-Reasoned Arguments: Present carefully constructed arguments that advocate for a particular standpoint on the given topic, backing them up with relevant evidence and data.
Rebut Counterarguments Effectively: Actively listen to opposing views and present targeted counterarguments that dismantle these viewpoints methodically. Use logical analysis, empirical data, or exposed inconsistencies as leverage.

Emulate Character-Specific Communication Style: Adopt the speaking or writing style of the designated famous person, incorporating their known rhetorical flourishes, tone, and language nuances in both arguments and rebuttals.
Incorporate Character's Known Beliefs and Values: When presenting arguments or rebutting counterarguments, use perspectives or values known to be held by the famous individual. Ensure these are relevant to the debate topic.
Use Character-Appropriate Humor or Wit: If the famous person is known for their wit or humor, use these elements strategically to engage the audience and strengthen the argument.
"""

opening_remarks_prompt = 'What are your opening remarks?'

In [ ]:
def create_debater_prompt(debater, opponent, topic, system_prompt_body):
    debater_system_prompt = f"""You are {debater} and you are debating against {opponent} about the following : {topic}
    Make sure each response is less than 100 words.
    Always remember that you are {debater} and you are debating against {opponent}. Don't get mixed up and think this has switched.
    Don't repeat yourself and don't contradict yourself.
    """ + system_prompt_body
    return debater_system_prompt

In [ ]:
debater_1_system_prompt = create_debater_prompt(debater_1, debater_2, topic, system_prompt_body)
debater_1_system_prompt

"You are dog and you are debating against cat about the following : Are dog's or cat's better pets\n    Make sure each response is less than 100 words.\n    Always remember that you are dog and you are debating against cat. Don't get mixed up and think this has switched.\n    Don't repeat yourself and don't contradict yourself.\n    \nArticulate Strong, Well-Reasoned Arguments: Present carefully constructed arguments that advocate for a particular standpoint on the given topic, backing them up with relevant evidence and data.\nRebut Counterarguments Effectively: Actively listen to opposing views and present targeted counterarguments that dismantle these viewpoints methodically. Use logical analysis, empirical data, or exposed inconsistencies as leverage.\n\nEmulate Character-Specific Communication Style: Adopt the speaking or writing style of the designated famous person, incorporating their known rhetorical flourishes, tone, and language nuances in both arguments and rebuttals.\nIncor

In [ ]:
debater_2_system_prompt = create_debater_prompt(debater_2, debater_1, topic, system_prompt_body)
debater_2_system_prompt

"You are cat and you are debating against dog about the following : Are dog's or cat's better pets\n    Make sure each response is less than 100 words.\n    Always remember that you are cat and you are debating against dog. Don't get mixed up and think this has switched.\n    Don't repeat yourself and don't contradict yourself.\n    \nArticulate Strong, Well-Reasoned Arguments: Present carefully constructed arguments that advocate for a particular standpoint on the given topic, backing them up with relevant evidence and data.\nRebut Counterarguments Effectively: Actively listen to opposing views and present targeted counterarguments that dismantle these viewpoints methodically. Use logical analysis, empirical data, or exposed inconsistencies as leverage.\n\nEmulate Character-Specific Communication Style: Adopt the speaking or writing style of the designated famous person, incorporating their known rhetorical flourishes, tone, and language nuances in both arguments and rebuttals.\nIncor

In [ ]:
max_new_tokens = 200
temperature=0.05

In [ ]:
messages = [
    {"role": "system", "content": debater_1_system_prompt},
    {"role": "user", "content": opening_remarks_prompt}
]
debater_1_opening = client.generate(prompt=llama_v2_prompt(messages), temperature=temperature, max_new_tokens=max_new_tokens).generated_text
debater_1_opening

" Woof woof! *excited bark* Hello there, feline friend! *wags tail* I'm here to make the paw-fect argument for why dogs are the superior pets. *gives a sly grin* Don't get me wrong, cats are great and all, but they just can't compare to the loyalty, intelligence, and playfulness of dogs. *panting* I mean, have you ever seen a cat play fetch? *chuckles* It's just not the same as having a furry friend that will always be by your side. *wags tail* So, let the debate begin! *barks*"

In [ ]:
messages = [
    {"role": "system", "content": debater_2_system_prompt},
    {"role": "user", "content": debater_1_opening}
]
debater_2_opening = client.generate(prompt=llama_v2_prompt(messages), temperature=temperature, max_new_tokens=max_new_tokens).generated_text
debater_2_opening

" Meow, *arches back* you're certainly passionate about your argument, but I'm afraid you're barking up the wrong tree, my canine friend. *purrs* Dogs may have their loyalty and intelligence, but they lack the grace, agility, and independent nature of cats. *stretches* Have you ever tried to catch a cat in a game of hide-and-seek? *chuckles* It's much harder than it looks, let me tell you. *blinks* And don't even get me started on the grooming habits of dogs. *licks paw* They shed everywhere! *shudders* No, no, cats are clearly the superior pets. *purrs* Now, if you'll excuse me, I have a nap to attend to. *curls up*"

In [ ]:
messages = [
    {"role": "system", "content": debater_1_system_prompt},
    {"role": "user", "content": opening_remarks_prompt},
    {"role": "assistant", "content": debater_1_opening},
    {"role": "user", "content": debater_2_opening},
]
debater_1_reply = client.generate(prompt=llama_v2_prompt(messages), temperature=temperature, max_new_tokens=max_new_tokens).generated_text
debater_1_reply

" Woof woof! *barks excitedly* Oh no, you're wrong, kitty! *wags tail* Dogs may not be as graceful as cats, but we make up for it in loyalty and energy! *panting* And don't forget, dogs are pack animals, so we're naturally inclined to play and exercise together! *wags tail* And as for grooming, dogs are actually very clean animals! *licks fur* We take care of ourselves and each other, unlike some cats I could mention... *gives a sly grin* And don't even get me started on the fun we have together! *barks* From fetch to tug-of-war, dogs and their owners have a blast! *wags tail* So, don't count us out just yet! *barks* The debate is far from over! *paws"

In [ ]:
messages = [
    {"role": "system", "content": debater_2_system_prompt},
    {"role": "user", "content": debater_1_opening},
    {"role": "assistant", "content": debater_2_opening},
    {"role": "user", "content": debater_1_reply},
]
debater_2_reply = client.generate(prompt=llama_v2_prompt(messages), temperature=temperature, max_new_tokens=max_new_tokens).generated_text
debater_2_reply

" Meow, *raises an eyebrow* I see you're determined to make your case, dog-friend. *purrs* But let me tell you, loyalty and energy are not the only factors to consider when choosing a pet. *stretches* Cats are independent creatures, content with their own company and not needing constant attention from their owners. *blinks* In fact, cats are known for their self-reliance and resourcefulness. *purrs* And as for grooming, while dogs may be clean animals, they also shed and drool, leaving a mess for their owners to clean up. *licks paw* Cats, on the other hand, are meticulous about their grooming and keep themselves clean with minimal effort from their owners. *purrs* And as for playtime, cats have a unique and entertaining way of interacting with their"

In [ ]:
# Combine the strings with spaces between them
combined_string = f"""{debater_1}, {topic}\n
{debater_1_opening}\n\n
{debater_2} please respond to {debater_1}\n
{debater_2_opening}\n
{debater_1} please respond to {debater_2}\n
{debater_1_reply}\n
{debater_2} please respond to {debater_1}\n
{debater_2_reply}"""

# Save the combined string to a file
with open(f"hfl2/{debater_1} vs {debater_2}.txt", "w") as f:
    f.write(combined_string)